In [ ]:
import os
from datetime import date, datetime
from zoneinfo import ZoneInfo
from typing import List, Dict, Tuple, Optional, Union
from decimal import Decimal

import django
# for easier visualization it is recommended to use pandas to render data...
# if pandas is not installed, you may install it with this command: pip install -U pandas
# pandas is not a dependency of django_ledger...
import pandas as pd

# Set your django settings module if needed...
os.environ['DJANGO_SETTINGS_MODULE'] = 'dev_env.settings'

# if using jupyter notebook need to set DJANGO_ALLOW_ASYNC_UNSAFE as "true"
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'

# change your working directory as needed...
os.chdir('../')

django.setup()

from django_ledger.models.entity import EntityModel
from django.contrib.auth import get_user_model
from django_ledger.io import roles

# Get Your Entity Administrator UserModel

In [ ]:
# change this to your preferred django username...
MY_USERNAME = 'ceo_user'
MY_PASSWORD = 'NeverUseMe|VeryInsecure!'
UserModel = get_user_model()

try:
    user_model = UserModel.objects.get(username__exact=MY_USERNAME)
except:
    user_model = UserModel(username=MY_USERNAME)
    user_model.set_password(MY_PASSWORD)
    user_model.save()

# Get or Create an Entity Model

In [ ]:
from django_ledger.models.entity import EntityModel

ENTITY_NAME = 'One Big Company, LLC'

entity_model = EntityModel.create_entity(
    name=ENTITY_NAME,
    admin=user_model,
    use_accrual_method=True,
    fy_start_month=1
)

entity_model

# Chart of Accounts

## Create a Default Chart of Accounts
- Newly created EntityModel do not have a default Code of Accounts yet.

In [ ]:
entity_model.has_default_coa()

In [ ]:
default_coa_model = entity_model.create_chart_of_accounts(
    assign_as_default=True,
    commit=True,
    coa_name='My QuickStart CoA'
)

default_coa_model

In [ ]:
default_coa_model

In [ ]:
entity_model.default_coa == default_coa_model

# Populate Entity with Random Data (Optional)

### Define a Start Date for Transactions

In [ ]:
START_DTTM = datetime(year=2022, month=10, day=1, tzinfo=ZoneInfo('UTC'))

- This action will populate the EntityModel with random data.
- It will populate a Code of Accounts using a default pre-defined list.
- This approach is for illustration, educational and testing purposes, not encouraged for new production entities.

In [ ]:
entity_model.populate_random_data(start_date=START_DTTM)

### EntityModel has now a Default Chart of Accounts

In [ ]:
entity_model.has_default_coa()

In [ ]:
default_coa_model = entity_model.get_default_coa()
default_coa_model

# Chart of Accounts (CoA)
- A Chart of Accounts is a user-defined list of accounts. 
- Each Entity Model must have at least one default Chart of Accounts.

## Django Ledger support multiple chart of accounts.

In [ ]:
another_coa_model = entity_model.create_chart_of_accounts(
    assign_as_default=False,
    commit=True,
    coa_name='My Empty Chart of Accounts'
)

In [ ]:
another_coa_model

# Accounts

## Default CoA Accounts

In [ ]:
default_coa_accounts_qs = entity_model.get_default_coa_accounts()
pd.DataFrame(default_coa_accounts_qs)

## Get CoA Accounts by CoA Model

In [ ]:
coa_accounts_by_coa_model_qs = entity_model.get_coa_accounts(coa_model=default_coa_model)
pd.DataFrame(coa_accounts_by_coa_model_qs)

No Accounts yet on this CoA...

In [ ]:
coa_accounts_by_coa_model_qs = entity_model.get_coa_accounts(coa_model=another_coa_model)
pd.DataFrame(coa_accounts_by_coa_model_qs)

## Get CoA Accounts by CoA Model UUID
- May pass UUID instance instead of ChartOF AccountsModel...

In [ ]:
coa_accounts_by_coa_uuid_qs = entity_model.get_coa_accounts(coa_model=default_coa_model.uuid)
pd.DataFrame(coa_accounts_by_coa_uuid_qs)

## Get CoA Accounts by CoA Model Slug
- If string is passed, will lookup by slug...

In [ ]:
coa_accounts_by_coa_slug_qs = entity_model.get_coa_accounts(coa_model=default_coa_model.slug)
pd.DataFrame(coa_accounts_by_coa_slug_qs)

## Get Accounts With Codes and CoA Model
- Assumes default CoA if no coa_model is passed...

In [ ]:
coa_accounts_by_codes_qs = entity_model.get_accounts_with_codes(code_list=['1010', '1050'])
pd.DataFrame(coa_accounts_by_codes_qs)

Empty ChartOfAccountModel...

In [ ]:
coa_accounts_by_codes_qs = entity_model.get_accounts_with_codes(
    code_list=['1010', '1050'], 
    coa_model=another_coa_model
)
pd.DataFrame(coa_accounts_by_codes_qs)

### Get All Accounts at Once

In [ ]:
coa_qs, coa_map = entity_model.get_all_coa_accounts()

A dictionary, CoA Model -> Account List.

In [ ]:
coa_map

In [ ]:
pd.DataFrame(coa_map[default_coa_model])

In [ ]:
pd.DataFrame(coa_map[another_coa_model])

## Create Account Model
- Creating AccountModel into empty "another_coa_model"...

In [ ]:
account_model = entity_model.create_account(
    coa_model=another_coa_model,
    code='1220',
    role=roles.ASSET_CA_INVENTORY,
    name='A new account created from the EntityModel API!',
    balance_type=roles.DEBIT,
    active=True
)

In [ ]:
account_model

In [ ]:
another_coa_accounts_qs = entity_model.get_coa_accounts(coa_model=another_coa_model)
pd.DataFrame(another_coa_accounts_qs)

# Basic Django Ledger Usage
- The LedgerModel name is whatever your heart desires.
- Examples:
    - A month.
    - A customer.
    - A vendor.
    - A project.
- The more ledgers are created, the more segregation and control over transactions is possible.

In [ ]:
ledger_model = entity_model.create_ledger(
    name='My October 2023 Ledger', 
    posted=True
)

## Create a Library

In [ ]:
from django_ledger.io.io_library import IOLibrary

library = IOLibrary(name='djangocon-2024-library')

## Create and Register a BluePrint

In [ ]:
from django_ledger.io.io_library import IOBluePrint

@library.register
def sale_blueprint(
        sale_amount: Union[int, float, Decimal],
        contribution_margin_percent: float,
        description: Optional[str] = None
) -> IOBluePrint:
    blueprint = IOBluePrint()
    cogs_amount = (1 - contribution_margin_percent) * sale_amount
    blueprint.debit(account_code='1010', amount=sale_amount, description=description)
    blueprint.credit(account_code='4010', amount=sale_amount, description=description)
    blueprint.credit(account_code='1200', amount=cogs_amount, description=description)
    blueprint.debit(account_code='5010', amount=cogs_amount, description=description)
    return blueprint

## Get a Cursor

In [ ]:
cursor = library.get_cursor(
    entity_model=entity_model, 
    user_model=user_model
)

## Dispatch Instructions

In [ ]:
# Option 1 - Use A Ledger Model
cursor.dispatch('sale_blueprint',
                ledger_model=ledger_model,
                sale_amount=34.45,
                contribution_margin_percent=0.13,
                description='Order ID: 123')

# Option 2- Create a New Ledger Model
cursor.dispatch('sale_blueprint',
                ledger_model='ledger-order-id-123',
                sale_amount=90.43,
                contribution_margin_percent=0.17,
                description='Order ID: 123')

## Commit Your Instructions
Not recommended to post both ledger and journal entries. Posted transactions will immediately hit the books.
**result** contains resulting ledger models, journal entries and transactions fro the committed 

In [ ]:
stub = cursor.commit(
    post_new_ledgers=True,
    post_journal_entries=True,
    je_timestamp=datetime(2023,12,2,12,10)
    # je_timestamp='2023-12-02 12:10'
)

### Get Financial Statement Report Data for Ledger Model

Balance Sheet

In [ ]:
bs_data = ledger_model.digest_balance_sheet(
    to_date=date(2023, 12, 31),
    entity_slug=entity_model
)

bs_data.get_balance_sheet_data()

Income Statement

In [ ]:
is_data = ledger_model.digest_income_statement(
    from_date=date(2023, 1, 1),
    to_date=date(2023, 12, 31),
    entity_slug=entity_model
)

is_data.get_income_statement_data()

Cash Flow Statement

In [ ]:
cfs_data = ledger_model.digest_cash_flow_statement(
    from_date=date(2023, 1, 1),
    to_date=date(2023, 12, 31),
    entity_slug=entity_model
)

cfs_data.get_cash_flow_statement_data()

All Statements in a Single Call

In [ ]:
fin_digest = ledger_model.digest_financial_statements(
    from_date=date(2023, 1, 1),
    to_date=date(2023, 12, 31),
    entity_slug=entity_model
)

statement_data = fin_digest.get_financial_statements_data()

In [ ]:
statement_data['balance_sheet']

In [ ]:
statement_data['income_statement']

In [ ]:
statement_data['cash_flow_statement']

# Financial Statement PDF Reports

## Set Up
- Must enable PDF support by installing dependencies via *pipenv*.
    - pipenv install --categories pdf

## Balance Sheet

In [ ]:
bs_report = entity_model.get_balance_sheet_statement(
    to_date=date(2022, 12, 31),
    save_pdf=True,
    filepath='./'
)
bs_data = bs_report.get_report_data()

In [ ]:
bs_data

### Balance Sheet Statement Raw Data

In [ ]:
bs_report.get_report_data()

## Income Statement

In [ ]:
ic_report = entity_model.get_income_statement(
    from_date=date(2022, 1, 1),
    to_date=date(2022, 12, 31),
    save_pdf=True,
    filepath='./'
)

ic_data = ic_report.get_report_data()

### Income Statement Raw Data

In [ ]:
ic_data

## Cash Flow Statement

In [ ]:
cf_report = entity_model.get_cash_flow_statement(
    from_date=date(2022, 1, 1),
    to_date=date(2022, 12, 31),
    save_pdf=True,
    filepath='./'
)

cf_data = cf_report.get_report_data()

### Cash Flow Statement Raw Data

In [ ]:
cf_data

## All Financial Statements Data in a single Call

In [ ]:
reports = entity_model.get_financial_statements(
    user_model=user_model,
    from_date=date(2022, 1, 1),
    to_date=date(2022, 12, 31),
    save_pdf=True,
    filepath='./'
)

In [ ]:
bs_data = reports.balance_sheet_statement.get_report_data()
ic_data = reports.income_statement.get_report_data()
cf_data = reports.cash_flow_statement.get_report_data()

In [ ]:
reports.income_statement.get_report_data()

In [ ]:
reports.cash_flow_statement.get_report_data()

# How To Extend the Ledger Model

In [ ]:
from django.db import models
from django_ledger.models import LedgerModel

class EmployeeModel(LedgerModel):
    
    dob = models.DateField()
    salary = models.DecimalField()
    first_name = models.CharField(max_length=50)
    last_name = models.CharField(max_length=50)
        
    def process_payroll(self, month):
        raise NotImplementedError()
    
    def send_payroll_report(self, month):
        raise NotImplementedError()